In [ ]:
#Importing the OptGene gene knockout framework and production envelope modules from cameo
from cameo.strain_design.heuristic.evolutionary_based import OptGene
from cameo import phenotypic_phase_plane
from cameo import load_model
from cameo.visualization.plotting.with_plotly import PlotlyPlotter

In [ ]:
#Loading the desired model
model = load_model('iJO1366_MEV.xml')

#Knock out MEP pathway, so only MEV is used
model.reactions.get_by_id("DXPRIi").knock_out()

#model = load_model('iJO1366_MEP.xml')

In [ ]:
#Setting the solver to gurobi
model.solver = 'gurobi'

In [ ]:
#Solving the FBA for the Wild Type model
wt_solution = model.optimize()
growth = wt_solution.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"]
caro_production = wt_solution.fluxes["DM_CRTY"]

In [ ]:
#Import set PlotlyPlotter as the plotter due to errors in the plot function
plotter = PlotlyPlotter()

#Set up the production envelope
p = phenotypic_phase_plane(model, variables=['BIOMASS_Ec_iJO1366_core_53p95M'], objective='DM_CRTY')
p.plot(plotter, points=[(growth, caro_production)])

In [ ]:
#Creating the OptGene model
optgene = OptGene(model)

In [ ]:
#Running the OptGene model to locate possible gene knockouts to improve flux through the beta-carotene sink reaction
#The OptGene is run to be growth coupled, so it aims to find reactions that would make the production of beta-carotene
#be tied with the growth of the organism
result = optgene.run(target=model.reactions.DM_CRTY, 
                     biomass=model.reactions.BIOMASS_Ec_iJO1366_core_53p95M,
                     substrate=model.metabolites.glc__D_e,
                     growth_coupled=True,
                     max_evaluations=20000,
                     population_size=200,
                     max_knockout=5,
                     plot=False)
result

In [ ]:
#If OptGene had found any possible knockouts this line would have been
#used to plot the production envelope for comparison between WT and mutants produced

#result.plot(plotter, 0)